In [1]:

import itertools
import numpy as np
import scipy.sparse as sp

In [2]:
#%%
import time
import argparse
import numpy as np
import pdb

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../')

from utils import load_data, accuracy,load_pokec
from models.FairGNN import FairGNN, FairGNN1

# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=2000,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.001,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=1e-5,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=128,
                    help='Number of hidden units of the sensitive attribute estimator')
parser.add_argument('--dropout', type=float, default=.5,
                    help='Dropout rate (1 - keep probability).')
parser.add_argument('--alpha', type=float, default=4,
                    help='The hyperparameter of alpha')
parser.add_argument('--beta', type=float, default=0.01,
                    help='The hyperparameter of beta')
parser.add_argument('--model', type=str, default="GAT",
                    help='the type of model GCN/GAT')
parser.add_argument('--dataset', type=str, default='nba',
                    choices=['pokec_z','pokec_n','nba'])
parser.add_argument('--num-hidden', type=int, default=64,
                    help='Number of hidden units of classifier.')
parser.add_argument("--num-heads", type=int, default=1,
                        help="number of hidden attention heads")
parser.add_argument("--num-out-heads", type=int, default=1,
                    help="number of output attention heads")
parser.add_argument("--num-layers", type=int, default=1,
                    help="number of hidden layers")
parser.add_argument("--residual", action="store_true", default=False,
                    help="use residual connection")
parser.add_argument("--attn-drop", type=float, default=.0,
                    help="attention dropout")
parser.add_argument('--negative-slope', type=float, default=0.2,
                    help="the negative slope of leaky relu")
parser.add_argument('--acc', type=float, default=0.688,
                    help='the selected FairGNN accuracy on val would be at least this high')
parser.add_argument('--roc', type=float, default=0.745,
                    help='the selected FairGNN ROC score on val would be at least this high')
parser.add_argument('--sens_number', type=int, default=200,
                    help="the number of sensitive attributes")
parser.add_argument('--label_number', type=int, default=500,
                    help="the number of labels")

args = parser.parse_known_args()[0]
args.cuda = not args.no_cuda and torch.cuda.is_available()
print(args)
#%%
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# Load datprint(args.dataset)
print(args.dataset)

if args.dataset != 'nba':
    if args.dataset == 'pokec_z':
        dataset = 'region_job'
    else:
        dataset = 'region_job_2'
    sens_attr = "region"
    predict_attr = "I_am_working_in_field"
    label_number = args.label_number
    sens_number = args.sens_number
    seed = 20
    path="../../dataset/pokec/"
    test_idx=False
else:
    dataset = 'nba'
    sens_attr = "country"
    predict_attr = "SALARY"
    label_number = 100
    sens_number = 50
    seed = 20
    path = "../../dataset/NBA"
    test_idx = True
print(dataset)

adj, features, labels, idx_train, idx_val, idx_test,sens,idx_sens_train = load_pokec(dataset,
                                                                                    sens_attr,
                                                                                    predict_attr,
                                                                                    path=path,
                                                                                    label_number=label_number,
                                                                                    sens_number=sens_number,
                                                                                    seed=seed,test_idx=test_idx)
print(len(idx_test))
#%%
import dgl

from utils import feature_norm
# G = dgl.DGLGraph()
# G.from_scipy_sparse_matrix(adj)
G = dgl.from_scipy(adj)
if dataset == 'nba':
    features = feature_norm(features)


def fair_metric(output,idx):
    val_y = labels[idx].cpu().numpy()
    idx_s0 = sens.cpu().numpy()[idx.cpu().numpy()]==0
    idx_s1 = sens.cpu().numpy()[idx.cpu().numpy()]==1

    idx_s0_y1 = np.bitwise_and(idx_s0,val_y==1)
    idx_s1_y1 = np.bitwise_and(idx_s1,val_y==1)

    pred_y = (output[idx].squeeze()>0).type_as(labels).cpu().numpy()
    parity = abs(sum(pred_y[idx_s0])/sum(idx_s0)-sum(pred_y[idx_s1])/sum(idx_s1))
    equality = abs(sum(pred_y[idx_s0_y1])/sum(idx_s0_y1)-sum(pred_y[idx_s1_y1])/sum(idx_s1_y1))

    return parity,equality
#%%
labels[labels>1]=1
if sens_attr:
    sens[sens>0]=1
# Model and optimizer

model = FairGNN1(nfeat = features.shape[1], args = args)
model.estimator.load_state_dict(torch.load("../checkpoint/GCN_sens_{}_ns_{}".format(dataset,sens_number)))
if args.cuda:
    # pdb.set_trace()
    G = G.to(torch.device('cuda:0'))
    model.cuda()
    features = features.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()
    sens = sens.cuda()
    idx_sens_train = idx_sens_train.cuda()



Using backend: pytorch
[15:45:42] /opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: /fs/classhomes/fall2021/cmsc742/cs742008/miniconda3/envs/py37/lib/python3.7/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.10.1.so: cannot open shared object file: No such file or directory


Namespace(acc=0.688, alpha=4, attn_drop=0.0, beta=0.01, cuda=True, dataset='nba', dropout=0.5, epochs=2000, fastmode=False, hidden=128, label_number=500, lr=0.001, model='GAT', negative_slope=0.2, no_cuda=False, num_heads=1, num_hidden=64, num_layers=1, num_out_heads=1, residual=False, roc=0.745, seed=42, sens_number=200, weight_decay=1e-05)
nba
nba
Loading nba dataset from ../../dataset/NBA
213


In [3]:
from sklearn.metrics import accuracy_score,roc_auc_score,recall_score,f1_score


# Train model
t_total = time.time()
best_result = {}
best_fair = 100

acc_test_list = []
roc_test_list = []
parity_list = []
equality_list = []

for epoch in range(args.epochs):
    t = time.time()
    model.train()
    model.optimize(G,features,labels,idx_train,sens,idx_sens_train)
    cov = model.cov
    cls_loss = model.cls_loss
    adv_loss = model.adv_loss
    model.eval()
    output,s,z = model(G, features)
    acc_val = accuracy(output[idx_val], labels[idx_val])
    roc_val = roc_auc_score(labels[idx_val].cpu().numpy(),output[idx_val].detach().cpu().numpy())


    acc_sens = accuracy(s[idx_test], sens[idx_test])
    
    parity_val, equality_val = fair_metric(output,idx_val)

    acc_test = accuracy(output[idx_test], labels[idx_test])
    roc_test = roc_auc_score(labels[idx_test].cpu().numpy(),output[idx_test].detach().cpu().numpy())
    parity,equality = fair_metric(output,idx_test)
    if acc_val > args.acc and roc_val > args.roc:
        acc_test_list.append(acc_test.item())
        roc_test_list.append(roc_test)
        parity_list.append(parity)
        equality_list.append(equality)    
        if best_fair > parity_val + equality_val :
            best_fair = parity_val + equality_val

            best_result['acc'] = acc_test.item()
            best_result['roc'] = roc_test
            best_result['parity'] = parity
            best_result['equality'] = equality

        print("=================================")

        print('Epoch: {:04d}'.format(epoch+1),
            'cov: {:.4f}'.format(cov.item()),
            'cls: {:.4f}'.format(cls_loss.item()),
            'adv: {:.4f}'.format(adv_loss.item()),
            'acc_val: {:.4f}'.format(acc_val.item()),
            "roc_val: {:.4f}".format(roc_val),
            "parity_val: {:.4f}".format(parity_val),
            "equality: {:.4f}".format(equality_val))
        print("Test:",
                "accuracy: {:.4f}".format(acc_test.item()),
                "roc: {:.4f}".format(roc_test),
                "acc_sens: {:.4f}".format(acc_sens),
                "parity: {:.4f}".format(parity),
                "equality: {:.4f}".format(equality))



print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

print('============performace on test set=============')
if len(best_result) > 0:
    print("Test:",
            "accuracy: {:.4f}".format(best_result['acc']),
            "roc: {:.4f}".format(best_result['roc']),
            "acc_sens: {:.4f}".format(acc_sens),
            "parity: {:.4f}".format(best_result['parity']),
            "equality: {:.4f}".format(best_result['equality']))
    print("Test:",
            "accuracy: {:.4f}".format(np.mean(acc_test_list)),
            "roc: {:.4f}".format(np.mean(roc_test_list)),
            "parity: {:.4f}".format(np.mean(parity_list)),
            "equality: {:.4f}".format(np.mean(equality_list)))
else:
    print("Please set smaller acc/roc thresholds")


Epoch: 0554 cov: 0.0035 cls: 0.4709 adv: 0.5751 acc_val: 0.6948 roc_val: 0.7455 parity_val: 0.0355 equality: 0.0194
Test: accuracy: 0.6948 roc: 0.7455 acc_sens: 0.8685 parity: 0.0355 equality: 0.0194
Epoch: 0584 cov: 0.0030 cls: 0.4994 adv: 0.5705 acc_val: 0.6948 roc_val: 0.7583 parity_val: 0.0590 equality: 0.0284
Test: accuracy: 0.6948 roc: 0.7583 acc_sens: 0.8685 parity: 0.0590 equality: 0.0284
Epoch: 0585 cov: 0.0074 cls: 0.4979 adv: 0.5236 acc_val: 0.6995 roc_val: 0.7586 parity_val: 0.0655 equality: 0.0194
Test: accuracy: 0.6995 roc: 0.7586 acc_sens: 0.8685 parity: 0.0655 equality: 0.0194
Epoch: 0586 cov: 0.0061 cls: 0.5305 adv: 0.5665 acc_val: 0.7042 roc_val: 0.7564 parity_val: 0.0799 equality: 0.0077
Test: accuracy: 0.7042 roc: 0.7564 acc_sens: 0.8685 parity: 0.0799 equality: 0.0077
Epoch: 0587 cov: 0.0061 cls: 0.4905 adv: 0.5775 acc_val: 0.6995 roc_val: 0.7549 parity_val: 0.0969 equality: 0.0077
Test: accuracy: 0.6995 roc: 0.7549 acc_sens: 0.8685 parity: 0.0969 equality: 0.0077


Epoch: 0882 cov: 0.0105 cls: 0.5060 adv: 0.5804 acc_val: 0.6948 roc_val: 0.7593 parity_val: 0.1116 equality: 0.0030
Test: accuracy: 0.6948 roc: 0.7593 acc_sens: 0.8685 parity: 0.1116 equality: 0.0030
Epoch: 0883 cov: 0.0029 cls: 0.4004 adv: 0.5597 acc_val: 0.6995 roc_val: 0.7593 parity_val: 0.0947 equality: 0.0030
Test: accuracy: 0.6995 roc: 0.7593 acc_sens: 0.8685 parity: 0.0947 equality: 0.0030
Epoch: 0915 cov: 0.0050 cls: 0.4347 adv: 0.5788 acc_val: 0.6901 roc_val: 0.7607 parity_val: 0.1358 equality: 0.1837
Test: accuracy: 0.6901 roc: 0.7607 acc_sens: 0.8685 parity: 0.1358 equality: 0.1837
Epoch: 0917 cov: 0.0024 cls: 0.3617 adv: 0.5668 acc_val: 0.6995 roc_val: 0.7606 parity_val: 0.1076 equality: 0.0151
Test: accuracy: 0.6995 roc: 0.7606 acc_sens: 0.8685 parity: 0.1076 equality: 0.0151
Epoch: 0918 cov: 0.0046 cls: 0.4037 adv: 0.5486 acc_val: 0.6948 roc_val: 0.7598 parity_val: 0.0882 equality: 0.0030
Test: accuracy: 0.6948 roc: 0.7598 acc_sens: 0.8685 parity: 0.0882 equality: 0.0030


Epoch: 1569 cov: 0.0007 cls: 0.3253 adv: 0.5542 acc_val: 0.6901 roc_val: 0.7495 parity_val: 0.0579 equality: 0.0159
Test: accuracy: 0.6901 roc: 0.7495 acc_sens: 0.8685 parity: 0.0579 equality: 0.0159
Epoch: 1570 cov: 0.0062 cls: 0.3449 adv: 0.5040 acc_val: 0.6995 roc_val: 0.7515 parity_val: 0.0604 equality: 0.0280
Test: accuracy: 0.6995 roc: 0.7515 acc_sens: 0.8685 parity: 0.0604 equality: 0.0280
Epoch: 1571 cov: 0.0047 cls: 0.4695 adv: 0.5730 acc_val: 0.6948 roc_val: 0.7515 parity_val: 0.0669 equality: 0.0400
Test: accuracy: 0.6948 roc: 0.7515 acc_sens: 0.8685 parity: 0.0669 equality: 0.0400
Epoch: 1572 cov: 0.0043 cls: 0.5913 adv: 0.5407 acc_val: 0.6995 roc_val: 0.7489 parity_val: 0.0709 equality: 0.0159
Test: accuracy: 0.6995 roc: 0.7489 acc_sens: 0.8685 parity: 0.0709 equality: 0.0159
Epoch: 1598 cov: 0.0074 cls: 0.3772 adv: 0.5874 acc_val: 0.6995 roc_val: 0.7460 parity_val: 0.0474 equality: 0.0159
Test: accuracy: 0.6995 roc: 0.7460 acc_sens: 0.8685 parity: 0.0474 equality: 0.0159


In [4]:
acc_test,roc_test,parity,equality

(tensor(0.6714, device='cuda:0', dtype=torch.float64),
 0.7545486663133724,
 0.12029495927801015,
 0.07616179001721168)

In [5]:
adj.shape

(403, 403)

In [6]:
z.shape

torch.Size([403, 64])

In [7]:
idx_train.shape

torch.Size([100])

In [8]:
np.intersect1d(idx_train.cpu(),idx_test.cpu()),np.intersect1d(idx_train.cpu(),idx_val.cpu()),np.intersect1d(idx_val.cpu(),idx_test.cpu()).shape

(array([], dtype=int64), array([], dtype=int64), (213,))

In [9]:
idx_test.shape

torch.Size([213])

In [10]:
idx_val.shape

torch.Size([213])

In [11]:
features.shape

torch.Size([403, 95])

In [12]:
s.shape

torch.Size([403, 1])

In [13]:
idx_sens_train.shape

torch.Size([50])

In [14]:
labels.shape

torch.Size([403])

In [15]:
adj.sum()

21645.0

In [16]:
adj!=adj.T

<403x403 sparse matrix of type '<class 'numpy.bool_'>'
	with 0 stored elements in Compressed Sparse Row format>

In [17]:
max(idx_train)

tensor(401, device='cuda:0')

In [18]:
G.number_of_nodes()

403

In [19]:
# def compute_loss(pos_score, neg_score):
#     # Margin loss
#     n_edges = pos_score.shape[0]
#     return (1 - pos_score.unsqueeze(1) + neg_score.view(n_edges, -1)).clamp(min=0).mean()

In [20]:
# class MLPPredictor(nn.Module):
#     def __init__(self, h_feats):
#         super().__init__()
#         self.W1 = nn.Linear(h_feats * 2, h_feats)
#         self.W2 = nn.Linear(h_feats, 1)

#     def apply_edges(self, edges):
       
#         h = torch.cat([edges.src['h'], edges.dst['h']], 1)
#         return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

#     def forward(self, g, h):
#         with g.local_scope():
#             g.ndata['h'] = h
#             g.apply_edges(self.apply_edges)
#             return g.edata['score']

In [80]:
def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).cuda()
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).cpu().numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).cpu().numpy()
    return roc_auc_score(labels, scores)

In [81]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super(MLPPredictor, self).__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def forward(self, edges1,edges2): # N,h (N,h)
        h = torch.cat([edges1, edges2], 1)
        out=self.W2(F.relu(self.W1(h))).squeeze(1)
        return out
        

In [82]:
g=G

In [164]:
noisy_estimate_factor=1.0
test_split=0.1

In [165]:
u, v = g.cpu().edges()

eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
nedges=int(len(eids)*noisy_estimate_factor)
test_size = int(nedges * test_split)
train_size = nedges - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:nedges]], v[eids[test_size:nedges]]

# Find all negative edges and split them for training and testing
adj1 = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj1_neg = 1 - adj1.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj1_neg != 0)

neg_eids = np.random.choice(len(neg_u), nedges// 2)
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:nedges ]], neg_v[neg_eids[test_size:nedges ]]


In [166]:
nedges

21645

In [167]:
train_size, test_size

(19481, 2164)

In [168]:
train_pos_u[0]

tensor(16)

In [169]:
train_pos_v[0]

tensor(369)

In [170]:
z[[1,3]].shape,z[train_pos_u].shape

(torch.Size([2, 64]), torch.Size([19481, 64]))

In [171]:
train_pos_edges1=z[train_pos_u].cuda().detach()
train_pos_edges2=z[train_pos_v].cuda().detach()
test_pos_edges1=z[test_pos_u].cuda().detach()
test_pos_edges2=z[test_pos_v].cuda().detach()
train_neg_edges1=z[train_neg_u].cuda().detach()
train_neg_edges2=z[train_neg_v].cuda().detach()
test_neg_edges1=z[test_neg_u].cuda().detach()
test_neg_edges2=z[test_neg_v].cuda().detach()

In [172]:
train_pos_edges1.shape,train_pos_edges2.shape, test_pos_edges1.shape,test_pos_edges2.shape, train_neg_edges1.shape,train_neg_edges2.shape, test_neg_edges1.shape,test_neg_edges2.shape


(torch.Size([19481, 64]),
 torch.Size([19481, 64]),
 torch.Size([2164, 64]),
 torch.Size([2164, 64]),
 torch.Size([8658, 64]),
 torch.Size([8658, 64]),
 torch.Size([2164, 64]),
 torch.Size([2164, 64]))

In [173]:
pred1 = MLPPredictor(64)
pred1.cuda()
pred1

MLPPredictor(
  (W1): Linear(in_features=128, out_features=64, bias=True)
  (W2): Linear(in_features=64, out_features=1, bias=True)
)

In [174]:
for name, param in pred1.named_parameters():
    if param.requires_grad:
        print(name, param.data.shape)

W1.weight torch.Size([64, 128])
W1.bias torch.Size([64])
W2.weight torch.Size([1, 64])
W2.bias torch.Size([1])


In [175]:
# ----------- 3. set up loss and optimizer -------------- #
# in this case, loss will in training loop
optimizer = torch.optim.Adam(pred1.parameters(), lr=0.002)

# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(40):
    # forward
    pred1.train()
    pos_score = pred1(train_pos_edges1, train_pos_edges2)
    neg_score = pred1(train_neg_edges1, train_neg_edges1)
    loss = compute_loss(pos_score, neg_score)
#     print('In epoch {}, loss: {}'.format(e, loss.data))
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss.data))

#     ----------- 5. check results ------------------------ #
    from sklearn.metrics import roc_auc_score
    with torch.no_grad():
        pos_score = pred1(test_pos_edges1, test_pos_edges2)
        neg_score = pred1(test_neg_edges1, test_neg_edges1)
        print('AUC', compute_auc(pos_score, neg_score))


pred1.eval()
# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred1(test_pos_edges1, test_pos_edges2)
    neg_score = pred1(test_neg_edges1, test_neg_edges1)
    print('AUC', compute_auc(pos_score, neg_score))

In epoch 0, loss: 0.6694367527961731
AUC 0.5986965117312022
AUC 0.6638258675827949
AUC 0.6808311566176144
AUC 0.6892614100334493
AUC 0.6976570694715407
In epoch 5, loss: 0.5751366019248962
AUC 0.7069094637164695
AUC 0.717354282478193
AUC 0.7282599271903539
AUC 0.7386156984908484
AUC 0.7492939198308056
In epoch 10, loss: 0.5411795377731323
AUC 0.7605100561703698
AUC 0.7737708674290439
AUC 0.7878207203405756
AUC 0.8027021099763906
AUC 0.8154491366026493
In epoch 15, loss: 0.5073724985122681
AUC 0.8246735567050817
AUC 0.8328836472131775
AUC 0.8399525635418768
AUC 0.8478333706321899
AUC 0.8565260684841176
In epoch 20, loss: 0.47538521885871887
AUC 0.8646208457330676
AUC 0.8722496719978406
AUC 0.8783025503876234
AUC 0.8832676403661324
AUC 0.8873576308335696
In epoch 25, loss: 0.444010853767395
AUC 0.8914269076229753
AUC 0.8954276370861108
AUC 0.8994240956877966
AUC 0.9032917878167698
AUC 0.90756371698197
In epoch 30, loss: 0.41496729850769043
AUC 0.9120310380585006
AUC 0.9157567240442666
AU

In [176]:
pos_score

tensor([2.4759, 0.0218, 2.2182,  ..., 0.7922, 1.1140, 0.7528], device='cuda:0')

In [177]:
neg_score

tensor([-0.4940,  0.2542, -0.0780,  ...,  0.2352,  0.1948, -0.2595],
       device='cuda:0')

In [178]:
torch.special.expit(pos_score)

tensor([0.9224, 0.5055, 0.9019,  ..., 0.6883, 0.7529, 0.6798], device='cuda:0')

In [179]:
torch.special.expit(neg_score)

tensor([0.3790, 0.5632, 0.4805,  ..., 0.5585, 0.5485, 0.4355], device='cuda:0')